In [1]:
from tdc.multi_pred import DDI
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

In [2]:
## label map

from tdc.utils import get_label_map
label_map = get_label_map(name = 'TWOSIDES', task = 'DDI', name_column = 'Side Effect Name')
label_map = {str(k): v for k, v in label_map.items()}
print(label_map)
print(len(label_map))

{'1024': 'hypermagnesemia', '767': 'retinopathy of prematurity', '79': 'atelectasis', '25': 'alkalosis', '85': 'Back Ache', '735': 'lung edema', '959': 'agitated', '255': 'abnormal movements', '9': 'Acidosis', '740': 'peliosis', '815': 'rupture of spleen', '56': 'Apnea', '247': 'Drug hypersensitivity', '328': 'flatulence', '1149': 'pain in throat', '442': 'allergies', '853': 'thrombocytopenia', '1226': 'bradycardia', '1132': 'lung infiltration', '391': 'Bleeding', '1072': 'hypoglycaemia neonatal', '1136': 'Gastrointestinal Obstruction', '433': 'hyperglycaemia', '679': 'peritonitis', '457': 'hypoglycaemia', '0': 'abdominal distension', '369': 'asystole', '143': 'cerebral infarct', '1143': 'hypoxia', '258': 'Difficulty breathing', '466': 'decreased body temperature', '658': 'adynamic ileus', '1155': 'sepsis', '1025': 'Hypomagnesaemia', '757': 'neonatal respiratory distress syndrome', '758': 'Acute Respiratory Distress Syndrome', '518': 'lipoma', '666': 'pelvic infection', '57': 'appendec

In [3]:
import json

json.dump(label_map, open('label_map.json', 'w'))

In [4]:
ts_data = DDI(name = 'TWOSIDES')
ts_split = ts_data.get_split()
twoside_drugs = set()

triplets = {
    'train': [],
    'valid': [],
    'test': []
}
all_triplets = []

for i in tqdm(range(len(ts_split['train']))):
    h = ts_split['train'].iloc[i]['Drug1_ID']
    t = ts_split['train'].iloc[i]['Drug2_ID']
    r = ts_split['train'].iloc[i]['Y']
    twoside_drugs.add(h)
    twoside_drugs.add(t)

    triplets['train'].append((h, t, r))
    all_triplets.append((h, t, r))
    
for i in tqdm(range(len(ts_split['valid']))):
    h = ts_split['valid'].iloc[i]['Drug1_ID']
    t = ts_split['valid'].iloc[i]['Drug2_ID']
    r = ts_split['valid'].iloc[i]['Y']
    twoside_drugs.add(h)
    twoside_drugs.add(t)

    triplets['valid'].append((h, t, r))
    all_triplets.append((h, t, r))
    
for i in tqdm(range(len(ts_split['test']))):
    h = ts_split['test'].iloc[i]['Drug1_ID']
    t = ts_split['test'].iloc[i]['Drug2_ID']
    r = ts_split['test'].iloc[i]['Y']
    twoside_drugs.add(h)
    twoside_drugs.add(t)

    triplets['test'].append((h, t, r))
    all_triplets.append((h, t, r))
print(len(twoside_drugs), len(all_triplets))

Found local copy...
Loading...
Done!
100%|██████████| 929888/929888 [03:46<00:00, 4107.36it/s]

645 4649441


In [5]:
print(f"Number of Drug: {len(twoside_drugs)}")
print(f"Len of train: {len(triplets['train'])}")
print(f"Len of valid: {len(triplets['valid'])}")
print(f"Len of test: {len(triplets['test'])}")
print(f"Len of edges: {len(all_triplets)}")

Number of Drug: 645
Len of train: 3254609
Len of valid: 464944
Len of test: 929888
Len of edges: 4649441


In [6]:
all_df = pd.DataFrame(all_triplets, columns =['Drug1', 'Drug2', 'DDI'])

print(all_df)

                Drug1         Drug2   DDI
0        CID000002173  CID000003345   767
1        CID000002173  CID000003345    25
2        CID000002173  CID000003345    85
3        CID000002173  CID000003345   735
4        CID000002173  CID000003345   959
...               ...           ...   ...
4649436  CID000000453  CID000004595   912
4649437  CID000005391  CID000068740  1281
4649438  CID000002909  CID000005978   289
4649439  CID000003937  CID000077999     2
4649440  CID000005212  CID006398525   201

[4649441 rows x 3 columns]


In [7]:
all_df['DDI'].value_counts().reset_index().to_csv('twosides_frequency.csv', index=False, header=None)

In [ ]:
# import pickle
# mfeat = []
# rfeat = []
# with open('id2drug_feat.pkl', 'rb') as f:
#     x = pickle.load(f, encoding='utf-8') 
    
# for z in x:
#     y = x[z]['Morgan']
#     mfeat.append(y)
#     y = x[z]['rdkit2d']
#     rfeat.append(y)
#     feat_dim = 1024

In [8]:
## filtered by id2drug.json and generate a map id2cid and cid2id
import json

# id2drug_json = json.load(open('id2drug.json'))


cid2id = {}

for (h,t,r) in all_triplets:
    if h not in cid2id.keys():
        cid2id[h] = len(cid2id)
    if t not in cid2id.keys():
        cid2id[t] = len(cid2id)
        

cids = cid2id.keys()
f = open('cid2id.json', 'w')
f.write(json.dumps(cid2id))
f.close()
print(cid2id)
print(cid2id.keys())
id2cid=dict([val,key] for key,val in cid2id.items())


{'CID000002173': 0, 'CID000003345': 1, 'CID000005206': 2, 'CID000009433': 3, 'CID000003929': 4, 'CID000150610': 5, 'CID000001302': 6, 'CID000005064': 7, 'CID000005267': 8, 'CID000010631': 9, 'CID000004601': 10, 'CID000005376': 11, 'CID000005090': 12, 'CID000005556': 13, 'CID000004946': 14, 'CID000130881': 15, 'CID000005391': 16, 'CID000005584': 17, 'CID000002802': 18, 'CID005381226': 19, 'CID000004212': 20, 'CID000060953': 21, 'CID000000596': 22, 'CID000005726': 23, 'CID000002522': 24, 'CID000003308': 25, 'CID000003405': 26, 'CID000060795': 27, 'CID000003446': 28, 'CID004183806': 29, 'CID000004107': 30, 'CID000004917': 31, 'CID000003161': 32, 'CID000071329': 33, 'CID000003823': 34, 'CID000003911': 35, 'CID000054454': 36, 'CID000002156': 37, 'CID000004411': 38, 'CID000002907': 39, 'CID000004192': 40, 'CID000083786': 41, 'CID000003348': 42, 'CID000000838': 43, 'CID000002617': 44, 'CID000001986': 45, 'CID000006691': 46, 'CID000004920': 47, 'CID000005478': 48, 'CID006398525': 49, 'CID00000

In [9]:
## generate relation and correspoding DDIs
twosides_freq = pd.read_csv("twosides_frequency.csv", header=None)

rel2id = {}
count = 0

for idx, row in twosides_freq.iterrows():
    # if idx >= 300 and idx < 1200:
    if idx < 1311:
        rel2id[row[0]] = count
        count += 1
print(len(rel2id))
print(rel2id)

1311
{464: 0, 37: 1, 258: 2, 600: 3, 692: 4, 315: 5, 648: 6, 238: 7, 74: 8, 926: 9, 963: 10, 321: 11, 154: 12, 1: 13, 1221: 14, 153: 15, 243: 16, 85: 17, 367: 18, 444: 19, 184: 20, 216: 21, 50: 22, 756: 23, 930: 24, 433: 25, 267: 26, 47: 27, 65: 28, 495: 29, 1230: 30, 649: 31, 895: 32, 201: 33, 189: 34, 840: 35, 929: 36, 853: 37, 688: 38, 1155: 39, 373: 40, 587: 41, 1286: 42, 81: 43, 245: 44, 304: 45, 1283: 46, 810: 47, 831: 48, 615: 49, 461: 50, 1115: 51, 878: 52, 215: 53, 139: 54, 596: 55, 884: 56, 959: 57, 391: 58, 344: 59, 32: 60, 430: 61, 1255: 62, 950: 63, 837: 64, 791: 65, 116: 66, 887: 67, 199: 68, 434: 69, 1216: 70, 725: 71, 1236: 72, 472: 73, 1118: 74, 257: 75, 662: 76, 1033: 77, 802: 78, 223: 79, 230: 80, 459: 81, 1116: 82, 653: 83, 364: 84, 369: 85, 457: 86, 61: 87, 0: 88, 735: 89, 736: 90, 1226: 91, 79: 92, 1257: 93, 1008: 94, 1143: 95, 345: 96, 1273: 97, 529: 98, 650: 99, 370: 100, 1028: 101, 678: 102, 442: 103, 76: 104, 342: 105, 207: 106, 163: 107, 371: 108, 1027: 109, 

In [ ]:
# dump the map between rel and id
# json.dump({str(k): str(v) for k, v in rel2id.items()}, open("new_data/rel2id.json", 'w'))
# json.dump({str(v): str(k) for k, v in rel2id.items()}, open("new_data/id2rel.json", 'w'))


In [10]:
filtered_triplets = []
for triplet in tqdm(all_triplets):
    if triplet[0] in cids and triplet[1] in cids and triplet[2] in rel2id:
        id1 = cid2id[triplet[0]]
        id2 = cid2id[triplet[1]]

        if id1 < id2:
            filtered_triplets.append((id1, id2, rel2id[triplet[2]]))
        else:
            filtered_triplets.append((id2, id1, rel2id[triplet[2]]))

100%|██████████| 4649441/4649441 [00:04<00:00, 1131831.81it/s]


In [11]:
print(f"{len(all_triplets)} -> {len(filtered_triplets)}")

4649441 -> 4649430


In [ ]:
# all_df = pd.DataFrame(filtered_triplets, columns =['Drug1', 'Drug2', 'DDI'])
# all_df['DDI'].value_counts().reset_index().to_csv('twosides_frequency_filtered.csv', index=False, header=None)
# print(all_df)

In [12]:
filtered_df = pd.DataFrame(filtered_triplets, columns = ['Drug1', 'Drug2', 'DDI'])

print(filtered_df)

         Drug1  Drug2   DDI
0            0      1  1224
1            0      1  1032
2            0      1    17
3            0      1    89
4            0      1    57
...        ...    ...   ...
4649425     79    239   212
4649426     16    399   343
4649427    118    304   292
4649428     50    144   492
4649429     49    356    33

[4649430 rows x 3 columns]


In [15]:
## convert format to one-hot
import numpy as np

max_ddi = filtered_df['DDI'].max()
print(max_ddi)
datas = {}

for idx, row in tqdm(filtered_df.iterrows()):
    pair_idx = (row['Drug1'], row['Drug2'])
    if pair_idx not in datas:
        datas[pair_idx] = np.zeros(1311, dtype=int)

    datas[pair_idx][row['DDI']] = 1

2266it [00:00, 22655.16it/s]

1310


4649430it [02:52, 26937.81it/s]


In [16]:
## Generate negative samples by corrupt head and tail entities
import random

with open('new_data/data.txt', 'w') as f:
    for pair, labels in tqdm(datas.items()):
        pos_str = f"{pair[0]}\t{pair[1]}\t{','.join(map(str, labels))}\t1\n"
        f.write(pos_str)

        rand1 = random.randint(0, len(cid2id)-1)
        rand2 = random.randint(0, len(cid2id)-1)
        neg_str = f"{rand1}\t{rand2}\t{','.join(map(str, labels))}\t0\n"
        f.write(neg_str)

100%|██████████| 63473/63473 [01:10<00:00, 899.12it/s] 


In [17]:
output_data = pd.read_csv('new_data/data.txt', sep='\t', header=None)
print(output_data)

          0    1                                                  2  3
0         0    1  1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...  1
1       573  199  1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...  0
2         2    3  0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...  1
3       152  102  0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...  0
4         4    5  0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...  1
...     ...  ...                                                ... ..
126941  389  421  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  0
126942  212  309  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  1
126943  583   23  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  0
126944  155  280  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  1
126945  111  478  0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...  0

[126946 rows x 4 columns]


In [18]:
## split data into train/valid/test
from sklearn.model_selection import train_test_split

# Split the data into train and remaining data
remaining_data, test_data = train_test_split(output_data, test_size=0.2, random_state=0)

# Split the remaining data into train and validation
train_data, valid_data = train_test_split(remaining_data, test_size=0.125, random_state=0)

# Print the lengths of the resulting datasets
print("Train data length:", len(train_data))
print("Validation data length:", len(valid_data))
print("Test data length:", len(test_data))

Train data length: 88861
Validation data length: 12695
Test data length: 25390


In [ ]:
train_data.to_csv('new_data/train.txt', sep='\t', header=None, index=False)
valid_data.to_csv('new_data/dev.txt', sep='\t', header=None, index=False)
test_data.to_csv('new_data/test.txt', sep='\t', header=None, index=False)

In [ ]:
## generate fingerprints
# from rdkit import Chem
# from rdkit.Chem import MACCSkeys, AllChem

# m = Chem.MolFromSmiles(mol_smiles[idx])
# fp = AllChem.GetHashedMorganFingerprint(m, 2, nBits=1024).ToList()
# fp_arr = np.array(fp)


## Drugbank

In [ ]:
drugbank_data = DDI(name = 'DrugBank')
drugbank_split = drugbank_data.get_split()

In [ ]:
all_drugbank = pd.concat([drugbank_split['train'], drugbank_split['valid'], drugbank_split['test']], axis=0)
print(all_drugbank)

In [ ]:
all_drugbank['Y'].value_counts().reset_index().to_csv("drugbank_frequency.csv", index=False, header=False)

In [ ]:
import json
id2drug = json.load(open('data/DDI_dataset/BioSNAP/id2drug.json','r',encoding="utf-8"))
cid2id = dict()
cid2db = dict()
for idx, drug in id2drug.items():
    cid2id[drug['cid']] = int(idx)
    db = drug['db']
    if db is not None:
        cid2db[drug['cid']] = db
cid_set = set(cid2id.keys())
print(len(cid_set), len(twoside_drugs), len(twoside_drugs.intersection(cid_set)))

In [ ]:
# relation mappings
from tdc.utils import get_label_map
relid2se = get_label_map(name = 'TWOSIDES', task = 'DDI', name_column = 'Side Effect Name')
relname2se = pd.read_csv('data/DDI_dataset/BioSNAP/side_effect_names.tsv', header=None, sep='\t')
se2relname = dict()
for i in range(len(relname2se)):
    relname, se = relname2se.iloc[i]
    se2relname[se] = relname
relid2name = dict()
for relid in relid2se:
    relid2name[relid] = se2relname[relid2se[relid]]

In [ ]:
relid2name

In [ ]:
rel_counts = defaultdict(int)
print(len(cid2id))
for triplet in all_triplets:
    h, t, r = triplet
    rel_counts[r] += 1
    if h in cid2id:
        h_id = cid2id[h]
    else:
        continue
        h_id = len(cid2id)
        cid2id[h] = h_id
    if t in cid2id:
        t_id = cid2id[t]
    else:
        continue
        t_id = len(cid2id)
        cid2id[t] = t_id

In [ ]:
rel_remaining = set()
N = 2790
for rel in relid2name:
    if rel_counts[rel] >= N:
        rel_remaining.add(rel)
print(len(rel_remaining), len(rel_counts))
rel2id = dict()
count = 0
for rel in rel_remaining:
    rel2id[rel] = count
    count += 1

In [ ]:
idx_triplets = defaultdict(set)
count = 0
for triplet in all_triplets:
    h, t, r = triplet
    if h in cid2id and t in cid2id and r in rel_remaining:
        h_id = cid2id[h]
        t_id = cid2id[t]
        r_id = rel2id[r]
        if h_id > t_id:
            h_id, t_id = t_id, h_id
        idx_triplets[(h_id, t_id)].add(r_id)
        count += 1
print(len(idx_triplets), count, count/(len(idx_triplets)*len(rel_remaining)))

In [ ]:
n_rel = len(rel2id)
with open('train.txt', 'w+') as f:
    for pair, rel in idx_triplets.items():
        h, t = pair
        out_str = "{}\t{}\t".format(h, t)
        rel_list = ["0"] * n_rel
        for r in rel:
            rel_list[r] = "1"
        out_str += ",".join(rel_list)
        out_str += "\t1\n"
        f.write(out_str)

## Plot frequency

In [ ]:
import pandas as pd
dataset = 'twosides'

plot_data = pd.read_csv(f"{dataset}_frequency.csv", header=None)

print(plot_data)

In [ ]:
y = plot_data[1][600:800]

print(y)
x = list(range(len(y)))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x, y)

plt.title(f"{dataset} Frequency")
plt.xlabel("Relation")
plt.ylabel("Frequency")

plt.ylim(0, 2000)

plt.show()